In [1]:
import requests
from bs4 import BeautifulSoup

def get_news(URL) :
  res = requests.get(URL)
  soup = BeautifulSoup(res.text, "html.parser")

  title = soup.select_one("h2#title_area span").text #제목
  date = soup.select_one("span.media_end_head_info_datestamp_time")['data-date-time'] #기사작성일시
  media = soup.select_one("a.media_end_head_top_logo img")['title'] #매체명 (예.한국경제)
  content = soup.select_one("div#newsct_article").text.replace("\n","") #기사원문

  return (title, date, media, content, URL)

In [2]:
get_news("https://n.news.naver.com/mnews/article/014/0005075348?sid=101")

('美 연방검찰, 테슬라의 머스크 특혜 수사',
 '2023-09-20 07:26:05',
 '파이낸셜뉴스',
 "\t\t\t  [파이낸셜뉴스]   일론 머스크 테슬라 최고경영자(CEO)가 15일(현지시간) 프랑스 파리 엘리제궁에서 에마뉘엘 마크롱 대통령 접견을 기다리고 있다. 월스트리트저널(WSJ)은 19일 소식통을 인용해 미국 연방검찰이 테슬라가 머스크 CEO에게 지난 수년간 상당한 특혜를 제공한 혐의를 잡고 수사하고 있다고 보도했다. 로이터연합  미국 연방검찰이 일론 머스크 최고경영자(CEO)에게 테슬라가 지난 수년간 상당한 특혜를 제공한 혐의에 대해 수사를 강화하고 있다.   수사 대상 기간도 이전에 알려졌던 것보다 더 이전인 2017년 이후로 올라갔다.   텍사스주 오스틴의 '유리집' 등을 포함해 테슬라 의사결정권자인 머스크가 자신의 주도로 테슬라에서 스스로에게 상당한 특혜를 준 것에 대한 형사기소로 이어질 가능성도 제기된다.   월스트리트저널(WSJ)은 19일(이하 현지시간) 소식통을 인용해 뉴욕남부연방지검이 테슬라, 그리고 머스크와 연관된 다른 업체 사이의 거래 내역 확보에 들어갔다고 보도했다. 소식통들에 따르면 검찰은 관련 대배심에 정보를 제출했다.   대배심에 제출된 새 자료에 따르면 연방검찰은 이전에 알려진 것보다 더 심각하게 머스크와 테슬라간 범죄혐의를 캐고 있다.   머스크의 유리집 건축 계획인 이른바 '프로젝트42'에 대해 검찰과 증권거래위원회(SEC)가 관심을 갖고 있다는 지난달 보도 내용보다 훨씬 깊이 검찰이 사건을 조사하고 있다고 WSJ은 전했다.   검찰이 현재 수사 중인 쟁점 가운데는 테슬라가 머스크에게 준 특혜를 적절히 공개했는지 여부다.   통상 기업 내·외부 변호사들이 이런 사안을 다루는 것이 일반적이지만 테슬라에서는 머스크가 주주들에게 정보를 공개할지 여부를 결정하곤 했다.   검찰이 테슬라의 머스크 특혜에만 집중하고 있는 것은 아니다.   테슬라 전기차 주행거리가 올바르게 표시되고 있는지 등도 수사 대상인 것으로 알려졌

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

keyword = "테슬라"
startdate="2022.08.21"
enddate="2023.09.21"

def get_news_list(keyword, startdate, enddate) :
  li = []
  h = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Referer' : 'https://search.naver.com/search.naver?where=news&query=%ED%85%8C%EC%8A%AC%EB%9D%BC&sm=tab_opt&sort=2&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Aall&is_sug_officeid=0&office_category=0&service_area=0', 'cookie':'NNB=KXDWSALR5HLWE; ASID=afc28e77000001821bee653700000062; _ga=GA1.2.852833469.1658355403; _ga_7VKFYR6RV1=GS1.1.1663030713.12.1.1663030726.47.0.0; NFS=2; m_loc=8e6c6458de8107ce6b301a2fdcaac47c270f10d32641f9fdfed1b5b1faac3e2c; NV_WETR_LAST_ACCESS_RGN_M="MDIxMzU1NTA="; NV_WETR_LOCATION_RGN_M="MDIxMzU1NTA="; recent_card_list=2936,3397,2717,3977; nx_ssl=2; nid_inf=-1453188587; NID_AUT=3O547E30xwR+LSPJWBh1H0BeJZ8z7w5GcYoFiB1oouz7XPFXgUYyapi0YaIPu/ed; NID_JKL=G/s4QbAdJVVTz69y/dBkR/9g00VQhbM8nxg71ZvVgyE=; NID_SES=AAABoKkgglCWz6aloghoOZ3uiyN2Gx8Ya6M3siOaeQCtWMn7TXgrPkganW/YVI931ONSrWmpB6IIM3p/mCMfueM9ekkTAuzM2mj8PfOoUbrV7BrzerfHGStcNmmb0QkiSuOy8AH1MwneQ7sJCTZBpPEfIbn9HUQ6S62sMy5oLJ0xqXedXxwQ4TsBa4+6Z6FWpVTIfmTzWMFt/M4pfpxYEbYAVBsfhLIsNPCjHLCrkoDdQPeUS949dDM9Xf8zpucBTRJrB6GwKaQDSeVWs+OgXngp1iusiktNnHZ7hEWDO5gyZCXkA7jV9njHKBDw6b58JD2La4eTPDBjq6xzUHaCAt+L7rGlpQ5FWHh1UF8XcyHGLdi2zQRHHSs9giWGIdbJ61akjlPZM/N8vL9zJZcw8qFmko0EY8RNmF1aUNE7qEcfaEyQNu/tlmIsUeYl5kPbZgL8fUiKMssPunn4ciffRsstEIcRbvtkVUdDKvr5QdQdf5J0o167t5vXBvctqzggQXUYOOBv8Cb9G9W5NcGibg9J3OCd7JMZTNGPV6gFAtga/WJL; page_uid=idY26sprvTVssudbzPwssssssI4-486433; _naver_usersession_=SXc2E3wZ6W9q/0SVHj+cMQ=='}

  for d in pd.date_range(startdate, enddate) :
    str_d = d.strftime("%Y.%m.%d")
    page = 1
    print(str_d)
    while True:
      start = (page-1)*10 + 1
      print(page)
      URL = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={0}&sort=2&photo=0&field=0&pd=3&ds={1}&de={2}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from{3}to{4},a:all&start={5}".format(keyword, str_d, str_d, str_d.replace(".",""), str_d.replace(".",""), start)

      res = requests.get(URL,headers = h)
      soup = BeautifulSoup(res.text, "html.parser")

      if soup.select_one(".api_noresult_wrap") :
        break

      news_list = soup.select("ul.list_news li")

      for item in news_list :
        if len(item.select("div.info_group a")) == 2 :
          li.append(get_news(item.select("div.info_group a")[1]['href']))
      page = page + 1

  return pd.DataFrame(li, columns=['title','date','media','content','url'])

In [4]:
get_news_list('테슬라', '2023.09.20', '2023.09.21')

2023.09.20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2023.09.21
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


,title,date,media,content,url
0,[천자칼럼] 웨스팅하우스,2023-09-20 00:32:12,한국경제,토머스 에디슨에게는 두 명의 강력한 라이벌이 있었다. 한 명은 그 유명한 니콜라 테...,https://n.news.naver.com/mnews/article/015/000...
1,“테슬라 공장 짓겠다”… 튀르키예도 유치전 뛰어들어,2023-09-20 03:04:05,동아일보,"뉴욕 간 에르도안, 머스크 면담위성통신 도입-AI 협력 등 논의머스크, 세 살배기 ...",https://n.news.naver.com/mnews/article/020/000...
2,X 유료화 추진·정상들 줄 세우기… 머스크 광폭행보,2023-09-20 04:08:29,국민일보,“스팸계정 등 봇 차단 목적” 밝혀월간 이용자 5억5000만명 달해올 들어 5개국 ...,https://n.news.naver.com/mnews/article/005/000...
3,연준 FOMC 앞두고 미증시 일제 하락…다우 0.31%↓(상보),2023-09-20 05:26:50,뉴스1,NYSE의 트레이더들. ⓒ 로이터=뉴스1(서울=뉴스1) 박형기 기자 = 미국 연준의...,https://n.news.naver.com/mnews/article/421/000...
4,내일 파월의 입 주목하는 시장…'미국판 컬리' 인스타카트 첫날 12%↑ [뉴욕마감],2023-09-20 05:34:48,머니투데이,뉴욕증시가 9월 기준금리 결정을 하루 앞두고 긴장감 속에서 소폭 하락했다. 전문가나...,https://n.news.naver.com/mnews/article/008/000...
...,...,...,...,...,...
256,"머스크 전기, 출간 첫 주 미국서 9만 2천부 판매고 '호조'",2023-09-21 21:04:15,MBC,사진제공:연합뉴스일론 머스크 테슬라 최고경영자의 전기가 미국에서 출간 첫 주에만 9...,https://n.news.naver.com/mnews/article/214/000...
257,단숨에 베스트셀러…머스크 전기 '불티',2023-09-21 21:25:27,한국경제TV,일론 머스크 테슬라 최고경영자(CEO)의 전기가 미국에서 출간 첫 주에만 9만부 넘...,https://n.news.naver.com/mnews/article/215/000...
258,[미리보는 이데일리신문]“美 고금리 내년까지”…새파래진 세계증시,2023-09-21 21:40:01,이데일리,\t\t\t[이데일리 김성진 기자] 다음은 22일자 이데일리 신문 주요 기사다.△1...,https://n.news.naver.com/mnews/article/018/000...
259,"머스크 전기, 출간 첫 주 베스트셀러 1위…“확실히 히트”",2023-09-21 22:02:01,매일경제,서점에 ‘일론 머스크’ 책이 쌓여 있다. [사진 제공 = 연합뉴스]일론 머스크 ...,https://n.news.naver.com/mnews/article/009/000...


In [5]:
import requests
from bs4 import BeautifulSoup

h = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Referer' : 'https://search.naver.com/search.naver?where=news&query=%ED%85%8C%EC%8A%AC%EB%9D%BC&sm=tab_opt&sort=2&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Aall&is_sug_officeid=0&office_category=0&service_area=0', 'cookie':'NNB=KXDWSALR5HLWE; ASID=afc28e77000001821bee653700000062; _ga=GA1.2.852833469.1658355403; _ga_7VKFYR6RV1=GS1.1.1663030713.12.1.1663030726.47.0.0; NFS=2; m_loc=8e6c6458de8107ce6b301a2fdcaac47c270f10d32641f9fdfed1b5b1faac3e2c; NV_WETR_LAST_ACCESS_RGN_M="MDIxMzU1NTA="; NV_WETR_LOCATION_RGN_M="MDIxMzU1NTA="; recent_card_list=2936,3397,2717,3977; nx_ssl=2; nid_inf=-1453188587; NID_AUT=3O547E30xwR+LSPJWBh1H0BeJZ8z7w5GcYoFiB1oouz7XPFXgUYyapi0YaIPu/ed; NID_JKL=G/s4QbAdJVVTz69y/dBkR/9g00VQhbM8nxg71ZvVgyE=; NID_SES=AAABoKkgglCWz6aloghoOZ3uiyN2Gx8Ya6M3siOaeQCtWMn7TXgrPkganW/YVI931ONSrWmpB6IIM3p/mCMfueM9ekkTAuzM2mj8PfOoUbrV7BrzerfHGStcNmmb0QkiSuOy8AH1MwneQ7sJCTZBpPEfIbn9HUQ6S62sMy5oLJ0xqXedXxwQ4TsBa4+6Z6FWpVTIfmTzWMFt/M4pfpxYEbYAVBsfhLIsNPCjHLCrkoDdQPeUS949dDM9Xf8zpucBTRJrB6GwKaQDSeVWs+OgXngp1iusiktNnHZ7hEWDO5gyZCXkA7jV9njHKBDw6b58JD2La4eTPDBjq6xzUHaCAt+L7rGlpQ5FWHh1UF8XcyHGLdi2zQRHHSs9giWGIdbJ61akjlPZM/N8vL9zJZcw8qFmko0EY8RNmF1aUNE7qEcfaEyQNu/tlmIsUeYl5kPbZgL8fUiKMssPunn4ciffRsstEIcRbvtkVUdDKvr5QdQdf5J0o167t5vXBvctqzggQXUYOOBv8Cb9G9W5NcGibg9J3OCd7JMZTNGPV6gFAtga/WJL; page_uid=idY26sprvTVssudbzPwssssssI4-486433; _naver_usersession_=SXc2E3wZ6W9q/0SVHj+cMQ=='}

URL = "https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query=%EC%82%BC%EB%B6%80%ED%86%A0%EA%B1%B4&oquery=%ED%85%8C%EC%8A%AC%EB%9D%BC&tqi=idYzedp0JXossC8LVxdssssssxw-085336&nso=so%3Ar%2Cp%3Afrom20230921to20230921%2Ca%3Aall&de=2023.09.21&ds=2023.09.21&mynews=0&office_category=0&office_section_code=0&office_type=0&pd=3&photo=0&service_area=0&sort=2"
print(URL)
res = requests.get(URL, headers = h)
soup = BeautifulSoup(res.text, "html.parser")

#soup

https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query=%EC%82%BC%EB%B6%80%ED%86%A0%EA%B1%B4&oquery=%ED%85%8C%EC%8A%AC%EB%9D%BC&tqi=idYzedp0JXossC8LVxdssssssxw-085336&nso=so%3Ar%2Cp%3Afrom20230921to20230921%2Ca%3Aall&de=2023.09.21&ds=2023.09.21&mynews=0&office_category=0&office_section_code=0&office_type=0&pd=3&photo=0&service_area=0&sort=2


In [6]:
import requests
from bs4 import BeautifulSoup


def get_blog(URL) :
  h = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Referer' : 'https://search.naver.com/search.naver?where=news&query=%ED%85%8C%EC%8A%AC%EB%9D%BC&sm=tab_opt&sort=2&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Aall&is_sug_officeid=0&office_category=0&service_area=0', 'cookie':'NNB=KXDWSALR5HLWE; ASID=afc28e77000001821bee653700000062; _ga=GA1.2.852833469.1658355403; _ga_7VKFYR6RV1=GS1.1.1663030713.12.1.1663030726.47.0.0; NFS=2; m_loc=8e6c6458de8107ce6b301a2fdcaac47c270f10d32641f9fdfed1b5b1faac3e2c; NV_WETR_LAST_ACCESS_RGN_M="MDIxMzU1NTA="; NV_WETR_LOCATION_RGN_M="MDIxMzU1NTA="; recent_card_list=2936,3397,2717,3977; nx_ssl=2; nid_inf=-1453188587; NID_AUT=3O547E30xwR+LSPJWBh1H0BeJZ8z7w5GcYoFiB1oouz7XPFXgUYyapi0YaIPu/ed; NID_JKL=G/s4QbAdJVVTz69y/dBkR/9g00VQhbM8nxg71ZvVgyE=; NID_SES=AAABoKkgglCWz6aloghoOZ3uiyN2Gx8Ya6M3siOaeQCtWMn7TXgrPkganW/YVI931ONSrWmpB6IIM3p/mCMfueM9ekkTAuzM2mj8PfOoUbrV7BrzerfHGStcNmmb0QkiSuOy8AH1MwneQ7sJCTZBpPEfIbn9HUQ6S62sMy5oLJ0xqXedXxwQ4TsBa4+6Z6FWpVTIfmTzWMFt/M4pfpxYEbYAVBsfhLIsNPCjHLCrkoDdQPeUS949dDM9Xf8zpucBTRJrB6GwKaQDSeVWs+OgXngp1iusiktNnHZ7hEWDO5gyZCXkA7jV9njHKBDw6b58JD2La4eTPDBjq6xzUHaCAt+L7rGlpQ5FWHh1UF8XcyHGLdi2zQRHHSs9giWGIdbJ61akjlPZM/N8vL9zJZcw8qFmko0EY8RNmF1aUNE7qEcfaEyQNu/tlmIsUeYl5kPbZgL8fUiKMssPunn4ciffRsstEIcRbvtkVUdDKvr5QdQdf5J0o167t5vXBvctqzggQXUYOOBv8Cb9G9W5NcGibg9J3OCd7JMZTNGPV6gFAtga/WJL; page_uid=idY26sprvTVssudbzPwssssssI4-486433; _naver_usersession_=SXc2E3wZ6W9q/0SVHj+cMQ=='}

  tmp = URL.split('/')
  Blog_URL = "https://blog.naver.com/PostView.naver?blogId={0}&logNo={1}".format(tmp[-2], tmp[-1])

  res = requests.get(Blog_URL, headers = h)
  soup = BeautifulSoup(res.text, "html.parser")

  if soup.select_one(".se-title-text") :
    title = soup.select_one(".se-title-text > p > span").text
    category = soup.select_one(".blog2_series > a").text
    nick = soup.select_one(".nick > a").text
    date = soup.select_one(".se_publishDate").text
    content = soup.select_one(".se-main-container").text.replace("\n","")
  elif soup.select_one("h3.se_textarea") :
    title = soup.select_one("h3.se_textarea").text
    category = soup.select_one(".blog2_series > a").text
    nick = soup.select_one(".nick > a").text
    date = soup.select_one(".se_publishDate").text
    content = soup.select_one(".se_doc_viewer").text.replace("\n","")
  else :
    return None

  return (title, category, nick, date, content, URL)

In [7]:
get_blog('https://blog.naver.com/over0322/223218145546')

('\n아이와 함께 읽으며 이야기를 나눌 수 있는 인물동화 - 일론 머스크, 별에 닿은 아이\n',
 '[책]기타',
 '려원맘 베키',
 '2023. 9. 21. 22:26',
 "[책]기타아이와 함께 읽으며 이야기를 나눌 수 있는 인물동화 - 일론 머스크, 별에 닿은 아이려원맘 베키 ・ 2023. 9. 21. 22:26URL 복사 이웃추가본문 기타 기능                   공유하기                신고하기📣 요즘 화제의 인물이죠! 저도 이름만 들었던 그 분을 이렇게 책으로 만나게 되었습니다.🔎 궁금했지만 자세히 알 수 없었던 그를 차근차근 알아가볼께요! 어린 시절부터 환경은 어떠했는지, 가정과 학교에서의 생활은 어떻게 지냈는지 쭉 나와있어요. 역시나 '천재소년'은 뭔가 달라도 다르군요!📚 매일매일 서점에서 살다시피하고 동생과 함께 실험하기를 좋아하고 어느 누가 강요하지 않아도 스스로 질문을 던져 생각하고.. 이게 바로 성공하는 자의 길이 아닐까 싶네요. 아이와 함께 자신의 진로를 위해 작은 목표를 세우며 읽어나가는 것도 좋을 것 같아요!⚖️ 아이를 키우면서 아이가 조금만 잘해도 '얘 천재인가?' 착각에 빠지게 되죠🤭 하지만 진짜 천재는 역시나 어렸을때부터 두각을 나타나네요. 우리아이도 이렇게 책을 읽었으면 하지만 그건 엄마의 욕심일 뿐이고.. 친구들하고 노는게 제일 좋은 아이일뿐이죠.. 그렇지만 책벌레 일론 머스크는 너무 똑똑한 나머지 학교에서는 친구들과 언제나 평화롭게 지낼 수는 없었던 듯 하네요.. 😥 이럴 때보면 세상은 참 공평하다.. 스스로에게 위로해봅니다😆😆⚠️ 이 글은 출판사 #아울북 으로부터 도서를 제공받아 작성되었습니다.#일론머스크별에닿은아이 #일론머스크어린이책 #테슬라 #초등추천도서 #위인전",
 'https://blog.naver.com/over0322/223218145546')

In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm as tqdm

keyword = "테슬라"
startdate = "20230921"
enddate = "20230921"

def get_blog_list(keyword, startdate, enddate) :
  start = 1
  URL = 'https://s.search.naver.com/p/blog/search.naver?where=blog&sm=tab_pge&api_type=1&query={0}&rev=44&start={1}&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so%3Add%2Cp%3Afrom{2}to{3}&nlu_query=%7B%22r_category%22%3A%2233+25%22%7D&dkey=0&source_query=&nx_search_query={0}&spq=0&_callback=viewMoreContents'.format(keyword, start, startdate, enddate)
  h = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Referer' : 'https://search.naver.com/search.naver?where=news&query=%ED%85%8C%EC%8A%AC%EB%9D%BC&sm=tab_opt&sort=2&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Aall&is_sug_officeid=0&office_category=0&service_area=0', 'cookie':'NNB=KXDWSALR5HLWE; ASID=afc28e77000001821bee653700000062; _ga=GA1.2.852833469.1658355403; _ga_7VKFYR6RV1=GS1.1.1663030713.12.1.1663030726.47.0.0; NFS=2; m_loc=8e6c6458de8107ce6b301a2fdcaac47c270f10d32641f9fdfed1b5b1faac3e2c; NV_WETR_LAST_ACCESS_RGN_M="MDIxMzU1NTA="; NV_WETR_LOCATION_RGN_M="MDIxMzU1NTA="; recent_card_list=2936,3397,2717,3977; nx_ssl=2; nid_inf=-1453188587; NID_AUT=3O547E30xwR+LSPJWBh1H0BeJZ8z7w5GcYoFiB1oouz7XPFXgUYyapi0YaIPu/ed; NID_JKL=G/s4QbAdJVVTz69y/dBkR/9g00VQhbM8nxg71ZvVgyE=; NID_SES=AAABoKkgglCWz6aloghoOZ3uiyN2Gx8Ya6M3siOaeQCtWMn7TXgrPkganW/YVI931ONSrWmpB6IIM3p/mCMfueM9ekkTAuzM2mj8PfOoUbrV7BrzerfHGStcNmmb0QkiSuOy8AH1MwneQ7sJCTZBpPEfIbn9HUQ6S62sMy5oLJ0xqXedXxwQ4TsBa4+6Z6FWpVTIfmTzWMFt/M4pfpxYEbYAVBsfhLIsNPCjHLCrkoDdQPeUS949dDM9Xf8zpucBTRJrB6GwKaQDSeVWs+OgXngp1iusiktNnHZ7hEWDO5gyZCXkA7jV9njHKBDw6b58JD2La4eTPDBjq6xzUHaCAt+L7rGlpQ5FWHh1UF8XcyHGLdi2zQRHHSs9giWGIdbJ61akjlPZM/N8vL9zJZcw8qFmko0EY8RNmF1aUNE7qEcfaEyQNu/tlmIsUeYl5kPbZgL8fUiKMssPunn4ciffRsstEIcRbvtkVUdDKvr5QdQdf5J0o167t5vXBvctqzggQXUYOOBv8Cb9G9W5NcGibg9J3OCd7JMZTNGPV6gFAtga/WJL; page_uid=idY26sprvTVssudbzPwssssssI4-486433; _naver_usersession_=SXc2E3wZ6W9q/0SVHj+cMQ=='}
  res = requests.get(URL, headers = h)
  total_page = int(res.text[28:].split('\"')[0])//30+1

  total_page = 10

  li = []

  for page in tqdm(range(1, total_page+1), desc="get_blog_list") :
    start = 30*(page-1)+1
    URL = 'https://s.search.naver.com/p/blog/search.naver?where=blog&sm=tab_pge&api_type=1&query={0}&rev=44&start={1}&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so%3Add%2Cp%3Afrom{2}to{3}&nlu_query=%7B%22r_category%22%3A%2233+25%22%7D&dkey=0&source_query=&nx_search_query={0}&spq=0&_callback=viewMoreContents'.format(keyword, start, startdate, enddate)
    res = requests.get(URL, headers = h)
    soup = BeautifulSoup(res.text.replace("\\",""), "html.parser")
    for item in soup.select('li div.total_area > a') :
      try :
        li.append(get_blog(item['href']))
      except :
        print(item['href'])

  return pd.DataFrame(li, columns=['title','category','nick','date','content', 'url'])

In [9]:
get_blog_list(keyword, startdate, enddate)

get_blog_list:   0%|          | 0/10 [00:00<?, ?it/s]

,title,category,nick,date,content,url
0,강남디테일링세차 나노디테일 추천,일상,푸드팔,2023. 9. 21. 23:59,23년9월20일 강남디테일링세차를 하기 위해 나노디테일에 방문을 했어요. NANO...,https://blog.naver.com/vnem8392/223218224579
1,"테슬라 400달러 리포트, 진짜 가능한 일일까?",어쩌다 어른경제,전인구경제연구소,2023. 9. 21. 23:57,최근 테슬라 주가가 10% 넘게 오른 일이 있었습니다. 그러면서 모건스탠리 아담 ...,https://blog.naver.com/fkldtt/223218223359
2,글로벌 자율주행 대표 기업 Top 10 (5) - 테슬라 Tesla,IT 트렌드,노아시스,2023. 9. 21. 23:54,"안녕하세요​.​지식의 오아시스, 노아시스​입니다.​지난 글에 이어오늘 살펴 볼다섯 ...",https://blog.naver.com/knoasis123/223218220005
3,넥스틸 PER 2배 영업이익률 27%,주식투자,창업의 지름신,2023. 9. 21. 23:54,이거 실화입니까? 지난 달인 2023년 8월 21일 상장하여 처음 거래된 넥스틸은 ...,https://blog.naver.com/bk4real/223218220719
4,[퓨런티어] 자율주행 시행착오는 퓨런티어의 성장동력 (f. 이베스트증권),관심기업,Tracker,2023. 9. 21. 23:52,▶ Subjective Summary자율주행 고도화로 핵심 부품들의 수요 성장세가 ...,https://blog.naver.com/arkiay88/223218218259
...,...,...,...,...,...,...
295,테슬라 썬팅 반사 필름은 어떠세요?,tinting,블락포디,2023. 9. 21. 14:53,수원 업템포 블락포디 스퀘어샵2열 프라이빗 글라스가 적용된 차종에는 어쩔 수 없지만...,https://blog.naver.com/menz380/223217756478
296,증시 이슈 24년 미래 성장산업 신재생 자율주행 로봇 반도체 방산 우주항공 위성관련주,경제공부,미소,2023. 9. 21. 14:51,"본 포스팅은 개인적인 경제공부로서 관심분야 이슈 스크랩입니다.​​안녕하세요, 공부하...",https://blog.naver.com/here---/223217754219
297,"테슬라 모델Y, 모던한 티나인 R70 과천 썬팅 시공 후기",프리미엄 썬팅,카스페이스 과천점,2023. 9. 21. 14:51,안녕하세요.과천 썬팅 전문카스페이스 과천점입니다. 오늘은 신차패키지가 아닌 썬팅 ...,https://blog.naver.com/guardwin01/223217754678
298,[2023년09월21일] 투자핵심뉴스 (5),투자관련 뉴스,민사랑,2023. 9. 21. 14:51,"英도 친환경 '유턴'…LG화학 장중 5% 급락, 2차전지株 휘청[핫종목]뉴스1|4...",https://blog.naver.com/hyungmjee/223217754723


In [10]:
list(range(1,48+1))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48]